# Chapter 7 — Multi-Agent: Loop

An agent **iterates** until a condition is met using `LoopAgent`.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Define a Loop Agent

A writer and reviewer iterate until the content meets quality standards.

In [ ]:
from google.adk.agents import Agent, LoopAgent, SequentialAgent

# Writer produces content
writer = Agent(
    name="writer",
    model="gemini-2.0-flash",
    description="Writes or revises content based on feedback.",
    instruction="""Write or revise content based on the user's request and any feedback from the reviewer.
Check session state for 'review_feedback' — if it exists, use it to improve your writing.
Store your output in 'current_draft'.""",
    output_key="current_draft",
)

# Reviewer evaluates and decides if we're done
reviewer = Agent(
    name="reviewer",
    model="gemini-2.0-flash",
    description="Reviews content and provides feedback or approves it.",
    instruction="""Review the draft in 'current_draft' from session state.
If the content is good enough (clear, well-structured, engaging), respond with exactly: APPROVED
If it needs improvement, provide specific feedback and store it in 'review_feedback'.
Be reasonable — approve after 1-2 rounds of revision at most.""",
    output_key="review_feedback",
)

# The inner sequential step: write then review
write_review_cycle = SequentialAgent(
    name="write_review_cycle",
    sub_agents=[writer, reviewer],
)

# Loop until the reviewer approves (max 3 iterations for safety)
loop_agent = LoopAgent(
    name="iterative_writer",
    sub_agents=[write_review_cycle],
    max_iterations=3,
)

print("Loop agent: write → review (repeat up to 3x)")

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=loop_agent, app_name="loop_demo", session_service=session_service)

session = await session_service.create_session(app_name="loop_demo", user_id="user1")

request = types.Content(
    role="user",
    parts=[types.Part(text="Write a short professional bio for a software engineer transitioning into AI/ML.")]
)

async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=request):
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text[:300]}...")
        print("---")

## Key Takeaways

- **`LoopAgent`**: Repeats sub-agents until a termination condition or max iterations
- **`max_iterations`**: Safety limit to prevent infinite loops
- Great for iterative refinement: draft → review → revise → review...